In [3]:
import os
os.environ["WANDB_DISABLED"] = "true"
os.environ["HF_HUB_ENABLE_HF_TRANSFER"] = "0"

import torch
import numpy as np
from dotenv import load_dotenv
from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    AutoModelForSequenceClassification,
    DataCollatorWithPadding,
    TrainingArguments,
    Trainer,
)
import evaluate

load_dotenv()
print(f"CUDA available: {torch.cuda.is_available()}")

CUDA available: True


In [4]:
raw_datasets = load_dataset("glue", "mrpc")
checkpoint = "bert-base-uncased"
tokenizer = AutoTokenizer.from_pretrained(checkpoint)

def tokenize_function(examples):
    return tokenizer(examples["sentence1"], examples["sentence2"], truncation=True)

tokenized_datasets = raw_datasets.map(tokenize_function, batched=True)
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [5]:
model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=2)
model = model.to("cuda")  # <-- Critical line

print(f"Model device: {next(model.parameters()).device}")  # Must show cuda:0

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model device: cuda:0


In [6]:
metric = evaluate.load("glue", "mrpc")

def compute_metrics(eval_preds):
    logits, labels = eval_preds
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [7]:
training_args = TrainingArguments(
    output_dir="../data/models/mrpc-model",
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    per_device_train_batch_size=32,
    fp16=True,
    report_to="none",
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["validation"],
    processing_class=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

print(f"Training device: {training_args.device}")  # Must show cuda:0
trainer.train()

Training device: cuda:0


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,No log,0.370327,0.843137,0.886926
2,No log,0.387839,0.840686,0.891486
3,No log,0.448561,0.845588,0.891938


TrainOutput(global_step=345, training_loss=0.31219918071359826, metrics={'train_runtime': 24.849, 'train_samples_per_second': 442.835, 'train_steps_per_second': 13.884, 'total_flos': 450668126729280.0, 'train_loss': 0.31219918071359826, 'epoch': 3.0})

In [8]:
# Examine model performance
eval_results = trainer.evaluate()
print(f"Eval results: {eval_results}")

Eval results: {'eval_loss': 0.37032729387283325, 'eval_accuracy': 0.8431372549019608, 'eval_f1': 0.8869257950530035, 'eval_runtime': 0.5485, 'eval_samples_per_second': 743.875, 'eval_steps_per_second': 92.984, 'epoch': 3.0}


In [9]:
# Eval on test set
test_results = trainer.evaluate(tokenized_datasets["test"])
print(f"Test results: {test_results}")

Test results: {'eval_loss': 0.38216152787208557, 'eval_accuracy': 0.8342028985507246, 'eval_f1': 0.8766177739430544, 'eval_runtime': 2.4389, 'eval_samples_per_second': 707.285, 'eval_steps_per_second': 88.564, 'epoch': 3.0}
